# Chemkin Input and Output
This notebook describes pMuTT's functionality to read and write Chemkin files. We will use the NH3 formation mechanism as a case study.

## Topics Covered
- Read species *ab-initio* data, reactions, and catalyst sites from a spreadsheet
- Write the thermdat, gas.inp, surf.inp, T_flow.inp, EAg.inp, EAs.inp, tube_mole.inp files

## Input Spreadsheet
All the data will be imported from the `NH3_Input_data.xlsx` file. There are four sheets:
1. `cat_sites` contains catalyst site properties for the adsorbed species
2. `refs` contains *ab-initio* and experimental data for a handful of gas species to calculate references
3. `species` contains *ab-initio* data for each specie
4. `reactions` contains elementary steps

The contents are displayed below

## Reading data
Before we can initialize our species, we need the catalytic sites and the references.

### Reading Catalytic Sites

In [1]:
import os
from pprint import pprint
from pathlib import Path
from pMuTT.io_.excel import read_excel
from pMuTT.chemkin import CatSite

# Find the location of Jupyter notebook
# Note that normally Python scripts have a __file__ variable but Jupyter notebook doesn't.
# Using pathlib can overcome this limiation
notebook_path = Path().resolve()
os.chdir(notebook_path)
excel_path = 'NH3_Input_data.xlsx'

cat_site_data = read_excel(io=excel_path, sheet_name='cat_sites')[0]
cat_site = CatSite(**cat_site_data)

# Print the properties of the catalyst site
pprint(cat_site.to_dict())

{'bulk_specie': 'RU(B)',
 'class': "<class 'pMuTT.chemkin.CatSite'>",
 'density': 12.2,
 'name': 'RU0001',
 'site_density': 2.1671e-09}


### Reading reference species

In [2]:
from pMuTT.empirical.references import Reference, References

references_data = read_excel(io=excel_path, sheet_name='refs')

# Convert data to Reference objects and put them in a list
refs_list = [Reference(**ref_data) for ref_data in references_data]

# Assign the Reference objects to a References object so offsets can be calculated
refs = References(references=refs_list)

# Print out the offsets calculated
print(refs.offset)

{'H': -129.34222830159834, 'N': -320.10077207763885}


### Reading species

In [3]:
from pMuTT.empirical.nasa import Nasa

# Range of data to fit the Nasa polynomials
T_low = 298. # K
T_high = 800. # K

species_data = read_excel(io=excel_path, sheet_name='species')
species = []
for specie_data in species_data:
    specie = Nasa.from_statmech(T_low=T_low, T_high=T_high, references=refs, **specie_data)
    # If the species is a surface species, assign the catalyst site specified above
    if specie.phase.lower() == 's':
        specie.cat_site = cat_site
        specie.n_sites = 1
    species.append(specie)

C:\Users\jonat\AppData\Roaming\Python\Python36\site-packages\scipy\stats\stats.py:1025: RuntimeWarning: invalid value encountered in double_scalars
  return a.std(axis) / a.mean(axis)


### Reading reactions

In [4]:
from pMuTT import pMuTT_list_to_dict
from pMuTT.reaction import ChemkinReaction, Reactions

# Convert list of Nasa polynomials to dictionary of Nasa polynomials
species_dict = pMuTT_list_to_dict(species)

reactions_data = read_excel(io=excel_path, sheet_name='reactions')
reactions_list = []
for reaction_data in reactions_data:
    reaction = ChemkinReaction.from_string(species=species_dict, **reaction_data)
    reactions_list.append(reaction)
reactions = Reactions(reactions=reactions_list)

## Writing Chemkin files
Now that we have all the required objects, we can write the output files.

### Writing thermdat

In [5]:
from pMuTT.io_.thermdat import write_thermdat

write_thermdat(filename='thermdat', nasa_species=species)

### Writing gas.inp and surf.inp

In [6]:
from pMuTT.io_ import chemkin as ck_io

ck_io.write_gas(filename='gas.inp', nasa_species=species, reactions=reactions, act_method_name='get_GoRT_act')
ck_io.write_surf(filename='surf.inp', nasa_species=species, reactions=reactions, act_method_name='get_GoRT_act')

### Writing T_flow.inp

In [7]:
# Conditions used to write files
T = [300., 400., 500.] # Temperature in K
P = [1., 2., 3.] # Pressure in atm
Q = [10., 20., 30.] # Standard volumetric flow rate in cm3
abyv= [100., 50., 25.] # Catalyst surface area to reactor volume in 1/cm

ck_io.write_T_flow(filename='T_flow.inp', T=T, P=P, Q=Q, abyv=abyv)

### Writing EAg.inp and EAs.inp

In [8]:
# Convert T_flow inputs into list of dictionaries that can be used by write_EA.
# In the future, this will be replaced by a function
conditions = []
for T_i, P_i, Q_i, abyv_i in zip(T, P, Q, abyv):
    condition = {
        'T': T_i,
        'P': P_i,
        'Q': Q_i,
        'abyv': abyv}
    conditions.append(condition)

ck_io.write_EA(filename='EAs.inp',
               reactions=reactions,
               write_gas_phase=False,
               act_method_name='get_GoRT_act',
               conditions=conditions)
ck_io.write_EA(filename='EAg.inp',
               reactions=reactions,
               write_gas_phase=True,
               act_method_name='get_GoRT_act',
               conditions=conditions)